In [1]:
from pathlib import Path
import bw2io
import bw2calc
import bw2data
from bw2io.importers.ilcd import ILCDImporter
import pandas as pd
from collections import Counter

In [2]:
print(bw2io.__version__)

(0, 9, 'DEV10')


In [3]:
bw2data.projects.set_current('ilcd_import')
bw2io.bw2setup()

Biosphere database already present!!! No setup is needed


## lcacommons data

In [4]:

path_to_example = Path('bw2io/data/examples/ilcd_examples/f29030e5-f681-31ae-9be8-d47f905cb0cb_3281fd83-15b3-4183-997d-dda9ea50dacd.zip')
lcacommons = ILCDImporter(dirpath= path_to_example,dbname='example_ilcd_lcacommons')


In [5]:
lcacommons.apply_strategies()

Applying strategy: rename_activity_keys
Applying strategy: get_activity_unit
Applying strategy: set_production_exchange
Applying strategy: convert_to_default_units
Applying strategy: assign_only_product_as_production
Applying strategy: map_to_biosphere3
Applying strategy: set_default_location
Applied 7 strategies in 0.04 seconds


In [6]:

lcacommons.match_database('biosphere3',fields=['database','code'])
lcacommons.match_database(fields=['database','code',])
lcacommons.statistics()

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
1 datasets
2128 exchanges
2112 unlinked exchanges
  Type biosphere: 1135 unique unlinked exchanges
  Type technosphere: 88 unique unlinked exchanges


(1, 2128, 2112)

In [7]:
lcacommons.data[0].get('name')

'coal extraction and processing - Northern Appalachia, WC, Underground'

In [8]:
lcacommons_df = pd.DataFrame(lcacommons.data[0]['exchanges'])

the data used in the ilcd 2 ecoinvent migration is the ones available in the GLAD database

Instead of using the ilcd to ecoinvent 3.7 mapping, lets use the `FEDEFL to ecoinvent`

In [9]:
fede_ei = pd.read_excel((Path.cwd() /'bw2io'/'data'/'FEDEFLv1.0.3-ecoinventEFv3.7.xlsx'))
fede_ei_mapping = fede_ei.set_index('SourceFlowUUID')['TargetFlowUUID'].dropna().to_dict()

In [10]:
for e in lcacommons.unlinked:
    code = fede_ei_mapping.get(e['uuid'])
    if code is not None:
        e['code'] = code

In [11]:
lcacommons.match_database('biosphere3',fields=['database','code'])

Applying strategy: link_iterable_by_fields


In [12]:
lcacommons.statistics()

1 datasets
2128 exchanges
1542 unlinked exchanges
  Type biosphere: 907 unique unlinked exchanges
  Type technosphere: 88 unique unlinked exchanges


(1, 2128, 1542)

the biosphere flows do not have a code... ?

In [13]:
for i,u in enumerate(lcacommons.unlinked):
    print(f"{u['type']} --- {u['name']} --- {u['uuid']}")

    if i > 10: break

technosphere --- 2,4-DINITROTOLUENE --- d3540cb8-a801-3913-9040-1619ab5b3b46
biosphere --- Methyl parathion --- 0cf75565-4164-3e75-b847-4974ebe979fc
biosphere --- Bicarbonate --- 1208f3d7-8e09-3e47-9fa8-cbef9b54253d
biosphere --- 2,4-D isopropyl ester --- 2fda9070-a8d8-3316-993e-42179ddaf687
biosphere --- Nitroglycerin --- 8ee2febb-ace4-38cc-b478-a62b5cf3ed1d
biosphere --- Benfluralin --- 076a10b9-d926-33f7-b146-10a1e778d9e7
biosphere --- Chromic(VI) acid --- 4717a8b5-2466-3dd1-b856-f639e8442e75
biosphere --- Pyrethrin I --- b61e086a-463f-3b36-b45d-649657392231
biosphere --- Cupric sulfate --- e9be3546-c54f-3ebc-96a2-a8897de5755f
biosphere --- Dimethipin --- 13a4f088-d48e-3c23-bd72-7b74d23a190e
biosphere --- (E)-Crotonic acid --- 9d718e3c-b210-3b88-8d2e-27982bcc69e8
technosphere --- CORROSIVE WASTE --- d5d165f8-aad3-3b82-a849-1d158c5d031d


In [14]:
Counter(lcacommons_df['type'])

Counter({'biosphere': 2024, 'technosphere': 103, 'production': 1})

In [15]:
lcacommons.drop_unlinked(True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds


In [16]:
postdrop_df = pd.DataFrame(lcacommons.data[0]['exchanges'])

In [17]:
Counter(postdrop_df['type'])

Counter({'biosphere': 570, 'technosphere': 15, 'production': 1})

In [18]:
postdrop_df.sample(3).T

,380,200,96
uuid,f7005ea1-d3f3-3df4-a992-60bfed8f12a8,362bf489-6818-38e9-94e5-e2bec93b7f06,35f65662-33c7-302d-854e-51a9cb08a35d
category_0,Elementary flows,Elementary flows,Elementary flows
category_1,emission,emission,emission
category_2,water,water,water
type,biosphere,biosphere,biosphere
refobj,93a60a56-a3c8-11da-a746-0800200b9a66,93a60a56-a3c8-11da-a746-0800200b9a66,93a60a56-a3c8-11da-a746-0800200b9a66
exchanges_internal_id,823,50,222
exchanges_name,Terbacil,Sodium chlorate,"2,4-D dimethylamine salt"
exchanges_direction,Output,Output,Output
amount,0.0,0.0,0.0


In [19]:
lcacommons.write_database()

Title: Writing activities to SQLite3 database:
  Started: 11/14/2022 20:13:35
  Finished: 11/14/2022 20:13:35
  Total time elapsed: 00:00:00
  CPU %: 41.80
  Memory %: 5.21
Created database: example_ilcd_lcacommons


Brightway2 SQLiteBackend: example_ilcd_lcacommons

In [20]:
commons_example = bw2data.Database('example_ilcd_lcacommons').random()

In [21]:
surprise = bw2data.Database('example_ilcd_lcacommons').random()
surprise

'coal extraction and processing - Northern Appalachia, WC, Underground' (kilogram, GLO, None)

In [22]:
gwp100 = ('IPCC 2013', 'climate change', 'GWP 100a')

In [23]:
lca = bw2calc.LCA({surprise:1},gwp100)
lca.lci()
lca.lcia()
lca.score

-155.96462451928224

the sign is a bit suspicious

## GLAD example

in the case of datasets from lca commons they seem.. wrong ?

In [24]:
path_to_GLAD_example = Path('bw2io/data/examples/ilcd-example.zip')

In [25]:
so = ILCDImporter(dirpath= path_to_GLAD_example, dbname='example_ilcd')

In [26]:
so.apply_strategies()
so.match_database('biosphere3',fields=['database','code'])
so.match_database(fields=['database','code'])
so.statistics()

Applying strategy: rename_activity_keys
Applying strategy: get_activity_unit
Applying strategy: set_production_exchange
Applying strategy: convert_to_default_units
Applying strategy: assign_only_product_as_production
Applying strategy: map_to_biosphere3
Applying strategy: set_default_location
Applied 7 strategies in 0.04 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
1 datasets
1957 exchanges
451 unlinked exchanges
  Type biosphere: 103 unique unlinked exchanges
  Type technosphere: 10 unique unlinked exchanges


(1, 1957, 451)

In [27]:
so.data[0].get('location')

'EU-28'

In [28]:
unlinked_df = pd.DataFrame(list(so.unlinked))

In [29]:
unlinked_df.loc[unlinked_df['type']=='technosphere',:]

,uuid,category_0,category_1,category_2,type,refobj,exchanges_internal_id,exchanges_name,exchanges_direction,amount,unit,flow property,name,database
3,fa1d0ee9-d657-4d0b-9ee4-7a0f5f46d462,None,None,None,technosphere,93a60a56-a3c8-11da-a746-0800200b9a66,1743,Spoil (deposited),Output,1.152937e-02,kilogram,Mass,Spoil (deposited),NaN
4,f0aeeb00-6d5b-4e9d-8e3b-169d2b2e56d4,None,None,None,technosphere,93a60a56-a3c8-11da-a746-0800200b9a66,1844,Low radioactive wastes,Output,1.139140e-05,kilogram,Mass,Low radioactive wastes,NaN
6,d7aa83f8-cc36-42d3-9475-2b241a4cc9ba,None,None,None,technosphere,93a60a56-a3c8-11da-a746-0800200b9a66,1031,Tailings (deposited),Output,7.875428e-03,kilogram,Mass,Tailings (deposited),NaN
7,a60b91d7-798b-4f76-973e-edc4ec8815a6,None,None,None,technosphere,93a60a56-a3c8-11da-a746-0800200b9a66,1906,Medium radioactive wastes,Output,5.183969e-06,kilogram,Mass,Medium radioactive wastes,NaN
10,8df3d2af-d7ac-4365-9b4a-17e28b9ca482,None,None,None,technosphere,93a60a56-a3c8-11da-a746-0800200b9a66,1080,Radioactive tailings,Output,5.976776e-04,kilogram,Mass,Radioactive tailings,NaN
12,62fafd63-a4e4-4ac2-adbb-1f838a2b6399,None,None,None,technosphere,93a60a56-a3c8-11da-a746-0800200b9a66,1580,Overburden (deposited),Output,2.078817e+00,kilogram,Mass,Overburden (deposited),NaN
17,3c4b0e5d-6500-4ada-9a2c-58e43ac96500,None,None,None,technosphere,93a60a56-a3c8-11da-a746-0800200b9a66,1589,Slag (deposited),Output,4.683707e-11,kilogram,Mass,Slag (deposited),NaN
21,26fd4ed1-a97f-4be3-a9ea-ca9273b48101,None,None,None,technosphere,93a60a56-a3c8-11da-a746-0800200b9a66,1672,High radioactive waste,Output,7.107064e-07,kilogram,Mass,High radioactive waste,NaN
23,24e51bc3-7272-4f3e-8d94-c6009a116c89,None,None,None,technosphere,93a60a56-a3c8-11da-a746-0800200b9a66,1891,Waste (deposited),Output,2.791233e-02,kilogram,Mass,Waste (deposited),NaN
37,03caafaa-5885-47d5-bef1-387d0d46b945,None,None,None,technosphere,93a60a56-a3c8-11da-a746-0800200b9a66,1376,Hazardous waste (deposited),Output,2.405829e-08,kilogram,Mass,Hazardous waste (deposited),NaN


In [30]:
so.drop_unlinked(True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds


In [31]:
e_df = pd.DataFrame(so.data[0]['exchanges'])

In [32]:
so.write_database()

Title: Writing activities to SQLite3 database:
  Started: 11/14/2022 20:13:38
  Finished: 11/14/2022 20:13:38
  Total time elapsed: 00:00:00
  CPU %: 76.20
  Memory %: 5.48
Created database: example_ilcd


Brightway2 SQLiteBackend: example_ilcd

In [33]:
tda = bw2data.Database('example_ilcd').random()

In [34]:
tda.as_dict()

{'treatment_standards_routes': 'phosgenation of diaminotoluene (TDA)',
 'mix_and_location_types': 'production mix, at plant',
 'functional_unit_flow_properties': 'liquid',
 'uuid': '84851553-b095-4e1c-a8b1-b100ce84b939',
 'general_comment': 'The main data source was a data collection from European producers of TDI. Primary data on gate-to-gate TDI production is derived from site-specific information for processes under operational control supplied by the participating companies of this study. Three different TDI producers with plants in three different European countries participated in the primary data collection. 100% of the European TDI production (EU-28) in 2018 is covered. The data for the upstream supply chain until the precursors are taken from the GaBi 2020 Databases of the software system GaBi 10. \r\nCombined elemental + mass allocation was applied for the production process of TDI as hydrogen chloride (HCl 100%) results as co-product.',
 'reference_year': '2018',
 'data_set_

In [35]:
lca = bw2calc.LCA({tda:1},gwp100)
lca.lci()
lca.lcia()
lca.score

3.143859395523786

## several ILCD files from GLAD

In [36]:
path_to_examples = Path('bw2io/data/examples/ilcd_examples')

In [37]:
assert path_to_examples.is_dir()

In [38]:
paths_to_examples = list(path_to_examples.glob('*zip'))

In [39]:
examples = []
for p in paths_to_examples:

    so = ILCDImporter(dirpath= p,dbname='example_ilcd')
    so.apply_strategies()
    so.match_database('biosphere3',fields=['database','code'])
    so.match_database(fields=['database','code'])
    so.statistics()

    examples.append(so)

Applying strategy: rename_activity_keys
Applying strategy: get_activity_unit
Applying strategy: set_production_exchange
Applying strategy: convert_to_default_units
Applying strategy: assign_only_product_as_production
Applying strategy: map_to_biosphere3
Applying strategy: set_default_location
Applied 7 strategies in 0.02 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
1 datasets
199 exchanges
178 unlinked exchanges
  Type biosphere: 133 unique unlinked exchanges
  Type technosphere: 10 unique unlinked exchanges
Applying strategy: rename_activity_keys
Applying strategy: get_activity_unit
Applying strategy: set_production_exchange
Applying strategy: convert_to_default_units
Applying strategy: assign_only_product_as_production
Applying strategy: map_to_biosphere3
Applying strategy: set_default_location
Applied 7 strategies in 0.02 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
1 datasets
605 excha

In [40]:
for e in examples:
    e.drop_unlinked(True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds
Applying strategy: drop_unlinked
Applied 1 strategies in 0.01 seconds
Applying strategy: drop_unlinked
Applied 1 strategies in 0.07 seconds
Applying strategy: drop_unlinked
Applied 1 strategies in 0.01 seconds
Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds
Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds


In [41]:
all_data = []
for e in examples:
    all_data = all_data + e.data

In [42]:
so.write_database(data=all_data)

Title: Writing activities to SQLite3 database:
  Started: 11/14/2022 20:13:46
  Finished: 11/14/2022 20:13:47
  Total time elapsed: 00:00:01
  CPU %: 64.40
  Memory %: 8.88
Created database: example_ilcd


Brightway2 SQLiteBackend: example_ilcd

In [43]:
ilcd_db = bw2data.Database('example_ilcd')

In [44]:
for act in ilcd_db:

    lca = bw2calc.LCA({act:1},gwp100)
    lca.lci()
    lca.lcia()
    lca.score
    print(f"{act['name']}  {lca.score}")

Polycarbonate  3.4241808028410965
Aluminium ingot (AlSi7Mg) sec.  0.2949287170350174
Short Chain Polyether Polyols (rigid)  3.1896583381743233
coal extraction and processing - Northern Appalachia, WC, Underground  0.0
Electricity - COAL - Southwest Power Pool  0.0
Cobalt, refined (metal)  1.5571662049552202
